# PEFT 库 LoRA 实战 - OpenAI Whisper-large-v2

本教程使用 LoRA 在`OpenAI Whisper-large-v2`模型上实现`语音识别(ASR)`任务的微调训练。

我们还结合了`int8` 量化进一步降低训练过程资源开销，同时保证了精度几乎不受影响。

In [1]:
!pip install torch>=2.1.2
!pip install transformers
!pip install ffmpeg
!pip install ffmpeg-python
!pip install timm
!pip install datasets
!pip install evaluate
!pip install scikit-learn
!pip install pandas
!pip install peft
!pip install accelerate
!pip install autoawq
!pip install optimum
!pip install auto-gptq
!pip install bitsandbytes>0.39.0
!pip install jiwer
!pip install soundfile>=0.12.1
!pip install librosa
!pip install langchain
!pip install gradio

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=c4f8a542c69cae05ec9c6a849ffa604253e2c0a7578d47e1c5a7e03b4575dbf9
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 全局参数设置

In [1]:
model_name_or_path = "openai/whisper-large-v2"
language = "Chinese (China)"
language_abbr = "zh-CN"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_11_0"

# 64 就基本极限的占满16GB的GPU显存了
batch_size=64

## 下载数据集 Common Voice

Common Voice 11.0 数据集包含许多不同语言的录音，总时长达数小时。

本教程以中文数据为例，展示如何使用 LoRA 在 Whisper-large-v2 上进行微调训练。

首先，初始化一个DatasetDict结构，并将训练集（将训练+验证拆分为训练集）和测试集拆分好，按照中文数据集构建配置加载到内存中：

In [2]:
#这段代码是在使用 datasets 库来加载和操作一个名为 common_voice 的数据集。datasets 库是由 Hugging Face 提供的，它允许用户以一种简单高效的方式加载、处理和管理数据集，特别是用于自然语言处理（NLP）和机器学习任务。t

# 从datasets库导入load_dataset函数。这个函数用于加载数据集。
from datasets import load_dataset
# 导入DatasetDict类，它用于创建和操作一个包含多个数据集分割（如训练集、测试集）的字典结构。
from datasets import load_dataset, DatasetDict

# 创建一个空的DatasetDict对象。这个对象将用于存储不同的数据集分割，如训练集和测试集。
common_voice = DatasetDict()

# 使用load_dataset函数加载训练和验证数据集，并将它们合并为一个数据集分割。
# 这里需要指定数据集的名称（dataset_name）、语言缩写（language_abbr）和数据集的分割（split="train+validation"）。
# 加载后的数据集被赋值给common_voice字典的"train"键。
common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation")
# 使用load_dataset函数加载测试数据集。
# 与上面相同，需要指定数据集的名称、语言缩写和数据集的分割（此处为"test"）。
# 加载后的数据集被赋值给common_voice字典的"test"键。
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test")
# 访问common_voice字典中"train"键对应的数据集分割，并获取该分割中的第一个样本。
# 这可以用于查看数据集中的样本格式，或进行其他操作。
common_voice["train"][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the ne

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 29056it [00:00, 145548.73it/s]


Generating validation split: 0 examples [00:00, ? examples/s]


Reading metadata...: 10581it [00:00, 133469.46it/s]


...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 15023it [00:00, 150215.46it/s]
Reading metadata...: 30826it [00:00, 154806.51it/s]
Reading metadata...: 46307it [00:00, 152008.25it/s]
Reading metadata...: 61514it [00:00, 148382.41it/s]
Reading metadata...: 76365it [00:00, 144413.71it/s]
Reading metadata...: 90825it [00:00, 141859.65it/s]
Reading metadata...: 105024it [00:00, 141512.22it/s]
Reading metadata...: 119533it [00:00, 142620.41it/s]
Reading metadata...: 133804it [00:00, 142527.97it/s]
Reading metadata...: 148063it [00:01, 142295.48it/s]
Reading metadata...: 162515it [00:01, 142966.70it/s]
Reading metadata...: 176815it [00:01, 139862.04it/s]
Reading metadata...: 191496it [00:01, 141922.47it/s]
Reading metadata...: 205773it [00:01, 142170.83it/s]
Reading metadata...: 220001it [00:01, 141807.64it/s]
Reading metadata...: 234190it [00:01, 140846.15it/s]
Reading metadata...: 248281it [00:01, 139150.92it/s]
Reading metadata...: 262204it [00:01, 138865.19it/s]
Reading met

Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 21302it [00:00, 147413.32it/s]
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'client_id': '95368aab163e0387e4fd4991b4f2d8ccfbd4364bf656c860230501fd27dcedf087773e4695a6cf5de9c4f1d406d582283190d065cdfa36b0e2b060cffaca977e',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/dcc5967c754d4c815fc005d6e297d84537028996cbcf6b34190517630cbc40b4/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/dcc5967c754d4c815fc005d6e297d84537028996cbcf6b34190517630cbc40b4/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([-6.82121026e-13, -2.27373675e-12, -2.27373675e-12, ...,
          1.21667399e-05,  3.23003678e-06, -2.43066324e-07]),
  'sampling_rate': 48000},
 'sentence': '性喜温暖润湿气候且耐寒。',
 'up_votes': 2,
 'down_votes': 0,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'zh-CN',
 'segment': ''}

## 预处理训练数据集


In [3]:
# 这段代码使用了 transformers 库来加载模型的特征提取器、分词器（tokenizer），以及一个综合处理器（processor）。transformers 库是由 Hugging Face 提供的，它提供了一系列预训练模型和工具，以便于进行自然语言处理（NLP）和机器学习任务。

# 从transformers库导入AutoFeatureExtractor, AutoTokenizer, 和 AutoProcessor类。
# 这些类允许自动识别和加载与给定模型名称或路径相匹配的预训练特征提取器、分词器和处理器。
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor

# 使用AutoFeatureExtractor的from_pretrained方法加载预训练的特征提取器。
# model_name_or_path参数指定了模型的名称或是存储模型的路径。特征提取器用于从输入数据中提取特征，特别是在图像和音频处理任务中。
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)

# 使用AutoTokenizer的from_pretrained方法加载预训练的分词器。
# 分词器用于将文本输入分割成模型能够理解的小单位（如单词、子词等）。
# 这里除了指定模型名称或路径，还可以指定语言和任务，以便加载适合特定任务的分词器配置。
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, language=language, task=task)

# 使用AutoProcessor的from_pretrained方法加载预训练的处理器。
# 处理器是特征提取器和分词器的结合，旨在简化文本和/或音频处理流程。
# 它自动处理输入数据的预处理和后处理，使得在不同类型的任务中使用模型变得更加容易。
# 同样，这里也可以指定模型名称或路径以及任务相关的配置，如语言和任务类型。
processor = AutoProcessor.from_pretrained(
    model_name_or_path, language=language, task=task)

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### 移除数据集中不必要的字段

In [4]:
#这行代码展示了如何使用 datasets 库中的 DatasetDict 对象的 remove_columns 方法来移除数据集中不需要的字段。这是在数据预处理阶段一个常见的步骤，用于简化数据集，只保留对后续分析或模型训练有用的信息。下面我将为这行代码添加注释，解释其工作原理和目的。

# 使用remove_columns方法从common_voice数据集中移除一系列指定的列。
# 这些列包括 "accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", 和 "up_votes"。
# 移除这些列可以减少数据集的大小和复杂性，特别是当这些信息对于即将进行的分析或模型训练不是必需的时候。
# remove_columns方法返回一个新的DatasetDict对象，其中已经移除了指定的列，原始的common_voice对象被更新为这个新对象。
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

In [5]:
# 这行代码的目的是访问更新后的 common_voice 数据集中的训练集部分（"train"），并获取该训练集中的第一个样本。

# 访问common_voice数据集的"train"部分，并获取这个训练集中的第一个样本。
# 这是在数据预处理或探索性数据分析阶段常见的操作，用于查看数据集的结构和样本的具体内容。
# 获取的样本是一个字典，其中的键是数据集中保留的字段名，值是对应的数据。
common_voice["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/dcc5967c754d4c815fc005d6e297d84537028996cbcf6b34190517630cbc40b4/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([-6.82121026e-13, -2.27373675e-12, -2.27373675e-12, ...,
          1.21667399e-05,  3.23003678e-06, -2.43066324e-07]),
  'sampling_rate': 48000},
 'sentence': '性喜温暖润湿气候且耐寒。'}

#### 降采样音频数据

查看`common_voice` 数据集介绍，你会发现其音频是以48kHz的采样率进行采样的.

而`Whisper`模型是在16kHZ的音频输入上预训练的，因此我们需要将音频输入降采样以匹配模型预训练时使用的采样率。

通过在音频列上使用`cast_column`方法，并将`sampling_rate`设置为16kHz来对音频进行降采样。

下次调用时，音频输入将实时重新取样：

In [6]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [7]:
common_voice["train"][0]

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/dcc5967c754d4c815fc005d6e297d84537028996cbcf6b34190517630cbc40b4/zh-CN_train_0/common_voice_zh-CN_33211332.mp3',
  'array': array([ 5.09317033e-11, -7.27595761e-12, -6.54836185e-11, ...,
         -5.96661994e-06,  2.71382887e-05,  1.29687978e-05]),
  'sampling_rate': 16000},
 'sentence': '性喜温暖润湿气候且耐寒。'}

### 整合以上数据处理为一个函数

该数据预处理函数应该包括：
- 通过加载音频列将音频输入重新采样为16kHZ。
- 使用特征提取器从音频数组计算输入特征。
- 将句子列标记化为输入标签。

In [8]:
# 这个函数 prepare_dataset 的目的是对数据集中的每个样本进行预处理，主要用于将音频数据转换为模型可处理的特征，同时将文本数据转换为对应的标签（通常是一个由单词或字符ID组成的序列）。

# 定义一个处理数据集样本的函数。
def prepare_dataset(batch):
    # 从批处理中的每个样本中获取音频数据。
    audio = batch["audio"]
    # 使用之前加载的特征提取器（feature_extractor）处理音频数据。
    # 这里的audio["array"]是音频的原始波形数据，audio["sampling_rate"]是音频的采样率。
    # 特征提取器将原始音频转换为模型可以理解的特征（如梅尔频谱图）。
    # 提取的特征被赋值给batch字典新的键"input_features"。
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # 使用之前加载的分词器（tokenizer）处理文本数据（batch中的"sentence"字段）。
    # 分词器将文本转换为一个由对应于文本中每个单词或字符的ID组成的序列。
    # 这个序列被用作模型的标签，赋值给batch字典的新键"labels"。
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    # 返回更新后的批处理数据。
    return batch

上面这个函数是在数据加载和处理流程中一个关键的步骤，它将原始数据转换为模型训练或评估所需的格式。通过对音频进行特征提取和对文本进行分词，它为后续的机器学习任务（如语音识别或自然语言处理任务）准备了数据。这种类型的数据预处理是实现高效和有效模型训练的基础。

In [9]:
# 这行代码使用了 datasets 库的 map 方法，对 common_voice 数据集的所有分割（例如训练集、测试集）进行了预处理，通过应用 prepare_dataset 函数来转换数据集中的每个样本。此外，它还移除了原始数据集中的所有列，只保留了预处理后的数据。

# 对common_voice数据集的每个分割（如训练集、测试集等）应用prepare_dataset函数。
# 这个函数对每个样本进行预处理，包括音频特征提取和文本的分词处理。
# map方法遍历数据集中的所有样本，对每个样本执行prepare_dataset函数定义的操作。
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"])

# 指定在映射操作后移除的列。这里移除的是数据集中所有原始列，意味着数据集将只包含
# prepare_dataset函数处理后添加的新列（如input_features和labels）。
# remove_columns=common_voice.column_names["train"]

Map:   0%|          | 0/39637 [00:00<?, ? examples/s]

Map:   0%|          | 0/10581 [00:00<?, ? examples/s]

执行上行代码报错：

Map:   0%|          | 0/39637 [00:00<?, ? examples/s]
---------------------------------------------------------------------------
OSError                                   Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py in _map_single(shard, function, with_indices, with_rank, input_columns, batched, batch_size, drop_last_batch, remove_columns, keep_in_memory, cache_file_name, writer_batch_size, features, disable_nullable, fn_kwargs, new_fingerprint, rank, offset)
   3455                             else:
-> 3456                                 writer.write(example)
   3457                         num_examples_progress_update += 1

24 frames
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

OSError                                   Traceback (most recent call last)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

OSError                                   Traceback (most recent call last)
/usr/local/lib/python3.10/dist-packages/fsspec/implementations/local.py in close(self)
    398
    399     def close(self):
--> 400         return self.f.close()
    401
    402     @property

OSError: [Errno 28] No space left on device

创建一个`DataCollator`类来将每个批次中的`attention_mask`填充到最大长度，并用`-100`替换填充值，以便在损失函数中被忽略。

然后初始化数据收集器的实例：

In [10]:
# 这段代码定义了一个名为 DataCollatorSpeechSeq2SeqWithPadding 的数据整理器类，用于为基于序列到序列（Seq2Seq）模型的语音处理任务准备批处理数据。这个数据整理器确保输入特征和标签都正确地进行了填充和处理，以便它们能够被模型以批处理的形式有效处理。
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


# 定义一个数据整理器类，使用dataclass装饰器以简化类定义
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    # 接收一个处理器作为参数，这个处理器既包含特征提取器也包含分词器
    processor: Any

    # 定义__call__方法，使得数据整理器的实例可以像函数一样被调用。
    # 接收的参数features是一个字典列表，字典中包含输入特征和标签。
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 从特征中提取输入特征，并准备进行填充
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        # 使用处理器的特征提取器进行填充，并转换为PyTorch张量
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # 从特征中提取标签，并准备进行填充
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 使用处理器的分词器进行填充，并转换为PyTorch张量
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # 使用掩码将填充位置的标签值设置为-100，这通常在训练时用于忽略计算损失的这些位置
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # 如果所有样本的第一个标签是开始符号（bos_token_id），则移除它
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        # 将处理后的标签添加到批处理字典中
        batch["labels"] = labels

        # 返回填充和处理后的批处理数据
        return batch

# 创建一个 DataCollatorSpeechSeq2SeqWithPadding 的实例，用于数据整理
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# 这个类对于准备用于序列到序列模型的语音处理任务的数据是非常重要的。它确保了输入特征和标签都被适当地填充并转换为模型所需的格式，同时处理了一些特殊情况，如标签的开始符号处理和填充位置的标签值设置，以便于模型的训练和评估。

## 训练模型

In [11]:
# 从transformers库导入AutoModelForSpeechSeq2Seq类。
from transformers import AutoModelForSpeechSeq2Seq

# 使用 from_pretrained 方法加载预训练的语音到文本模型。
# model_name_or_path 参数指定了模型的名称或者是模型的路径。
# load_in_8bit 参数启用了8位量化，这有助于减少模型加载时的内存消耗。
# device_map="auto" 参数指示模型应自动分布到可用的设备上（如GPU），以优化计算性能。
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [12]:
# 修改模型配置，设置forced_decoder_ids为None。
# 这可能是为了确保在解码过程中不强制使用特定的解码器ID，让模型能够自由地基于上下文进行解码。
model.config.forced_decoder_ids = None
# 修改模型配置，设置suppress_tokens为空列表。
# 这表示在解码过程中没有特定的令牌会被主动抑制或排除。
model.config.suppress_tokens = []

为了准备模型进行int8量化，使用 `prepare_model_for_int8_training` 函数来处理模型：
- 将所有非int8模块转换为完全精度（fp32）以保持稳定性
- 在输入嵌入层上添加前向钩子，计算输入隐藏状态的梯度
- 启用渐变检查点以进行更高效的内存训练

In [13]:
# 从peft库导入 prepare_model_for_int8_training 函数。
# peft（可能代表 Post-Training Quantization for Efficient Fine-Tuning或类似的意思）库
# 提供了工具和方法来优化模型，特别是通过量化来减少模型的内存和计算需求。
from peft import prepare_model_for_int8_training
# 使用 prepare_model_for_int8_training 函数准备模型进行Int8训练。
# 这个过程可能包括模型权重的量化，以及可能的其他优化，以使模型在保持性能的同时，
# 减少其对计算资源的需求。
model = prepare_model_for_int8_training(model)

# 以上这个过程是在模型的微调或训练前进行的优化步骤，目的是通过量化技术减小模型大小和加速模型的计算，
# 这在资源有限或对延迟敏感的应用场景中特别有用。
# 量化将模型权重和激活从浮点数转换为整数，减少了模型的内存占用，并可能加速推理和训练过程。
# 不过，值得注意的是，peft 库和 prepare_model_for_int8_training 函数在标准的 transformers 库
# 或PyTorch环境中并不直接可用，这可能是一个自定义的函数或库，或者需要额外安装的依赖。
# 在实际应用中，应该查找相关的库或工具，以及如何正确安装和使用它们的指南。

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:143: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [14]:
# 这段代码涉及到使用 LoraConfig 类来创建一个配置对象，该对象用于初始化或配置 LoRA（Low-Rank Adaptation）模型的参数。LoRA 是一种模型参数化技术，通过在预训练模型的特定层中添加低秩矩阵来实现参数的有效微调。这种方法特别适用于NLP任务，能够在极少的额外参数下实现显著的性能提升。

# 从peft库导入LoraConfig, PeftModel, LoraModel, 和get_peft_model。
# 这些组件可能是专为参数效率调整（如LoRA）设计的自定义库或工具。
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

# 创建一个LoraConfig对象，用于指定LoRA的配置参数。
config = LoraConfig(
    r=8, # 设置LoRA适配器的秩为8。秩是指LoRA矩阵分解中的中间维度，较小的秩意味着较少的参数。
    lora_alpha=64, # 设置LoRA的放大因子为64。这个参数控制了LoRA矩阵的缩放程度。
    target_modules=["q_proj", "v_proj"], # 指定要在其中应用LoRA适配器的模型模块。
    lora_dropout=0.05, # 设置LoRA层中的dropout率为0.05，以防过拟合。
    bias="none") # 指定LoRA层不使用偏置项。这是LoRA参数配置的一部分。

# 这个配置主要用于微调或适配阶段，通过在模型的特定模块（如Transformer的q_proj和v_proj层）中引入LoRA适配器来进行。这样做可以在不显著增加模型大小和计算成本的情况下，有效地提升模型的性能。LoraConfig 类提供了一种灵活的方式来指定LoRA适配器的各种参数，包括其秩（r）、放大因子（lora_alpha）、目标模块（target_modules）、dropout率（lora_dropout）和是否使用偏置项（bias）。

# 请注意，peft 库和其中的 LoraConfig, PeftModel, LoraModel, get_peft_model 可能是特定于某些研究项目或尚未广泛采用的工具，因此在实际应用中可能需要根据具体情况寻找相应的文档或实现。

In [15]:
# 这段代码展示了如何使用一个自定义函数 get_peft_model 将先前创建的 LoRA 配置应用于一个模型，并随后调用模型的 print_trainable_parameters 方法来打印出模型中可训练参数的详情。这里假设 get_peft_model 函数负责将 LoRA 适配器整合到指定的模型中，并返回一个增强后的模型实例。

# 使用get_peft_model函数应用LoRA配置到现有模型上。
# 这个函数接收原始模型和LoRA配置作为参数，并返回一个经过修改（例如添加了LoRA适配器）的模型。
model = get_peft_model(model, config)
# 调用模型的print_trainable_parameters方法打印出模型中所有可训练参数的详细信息。
# 这可能包括参数的名称、尺寸，以及它们是否被冻结（不更新）。
model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 1,547,237,120 || trainable%: 0.25414074863974306


### 演示需要，只训练了100 steps。建议同学改为默认的 3个 epochs 完整训练一个中文语音识别模型。

In [16]:
from transformers import Seq2SeqTrainingArguments

# 设置序列到序列模型训练的参数
training_args = Seq2SeqTrainingArguments(
    output_dir="models/whisper-large-v2-asr-int8",  # 指定模型输出和保存的目录
    per_device_train_batch_size=batch_size,  # 每个设备上的训练批量大小
    gradient_accumulation_steps=1,  # 梯度累积步数，在每次优化器步骤之前累积的更新步数
    learning_rate=1e-3,  # 学习率
    warmup_steps=50,  # 在训练初期增加学习率的步数，有助于稳定训练
    max_steps=100, # 训练总步数
    # num_train_epochs=3,  # 训练的总轮数 -- 注释放开，跑一个中文语音识别模型完整的训练。
    # evaluation_strategy="epoch",  # 设置评估策略，这里是在每个epoch结束时进行评估 -- 注释放开，跑一个中文语音识别模型完整的训练。
    fp16=True,  # 启用混合精度训练，可以提高训练速度，同时减少内存使用
    per_device_eval_batch_size=batch_size,  # 每个设备上的评估批量大小
    generation_max_length=128,  # 生成任务的最大长度
    logging_steps=25,  # 指定日志记录的步骤，用于跟踪训练进度
    remove_unused_columns=False,  # 是否删除不使用的列，以减少数据处理开销
    label_names=["labels"],  # 指定标签列的名称，用于训练过程中
)
# max_steps=100 100个步数，Tesela T4 训练时间在1个半小时左右

#### 训练过程保存状态的回调，长时期训练建议使用

In [17]:
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from transformers import Seq2SeqTrainer, TrainerCallback, Seq2SeqTrainingArguments, TrainerState, TrainerControl

class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: Seq2SeqTrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

In [18]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False

In [19]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
25,2.417700
50,0.571500
75,0.409900
100,0.391600


TrainOutput(global_step=100, training_loss=0.9476741504669189, metrics={'train_runtime': 1551.7978, 'train_samples_per_second': 4.124, 'train_steps_per_second': 0.064, 'total_flos': 1.362453331968e+19, 'train_loss': 0.9476741504669189, 'epoch': 0.16})

### 保存 LoRA 模型

In [20]:
model.save_pretrained("models/whisper-large-v2-asr-int8")

### 保存分词器

In [27]:
tokenizer.save_pretrained("models/whisper-large-v2-asr-int8/tokenizer")  # 保存分词器

('models/whisper-large-v2-asr-int8/tokenizer/tokenizer_config.json',
 'models/whisper-large-v2-asr-int8/tokenizer/special_tokens_map.json',
 'models/whisper-large-v2-asr-int8/tokenizer/vocab.json',
 'models/whisper-large-v2-asr-int8/tokenizer/merges.txt',
 'models/whisper-large-v2-asr-int8/tokenizer/normalizer.json',
 'models/whisper-large-v2-asr-int8/tokenizer/added_tokens.json',
 'models/whisper-large-v2-asr-int8/tokenizer/tokenizer.json')

#### save_pretrained

In [35]:
trainer.model.save_pretrained("./")

In [36]:
!pwd

/content


In [37]:
!ls -lh

total 16M
-rw-r--r-- 1 root root  322 Feb  7 02:24 '=0.12.1'
-rw-r--r-- 1 root root  577 Feb  7 02:23  0.39.0
-rw-r--r-- 1 root root 1.1K Feb  7 02:22 '=2.1.2'
-rw-r--r-- 1 root root  720 Feb  7 04:50  adapter_config.json
-rw-r--r-- 1 root root  16M Feb  7 04:50  adapter_model.safetensors
drwxr-xr-x 4 root root 4.0K Feb  7 04:41  data
drwx------ 5 root root 4.0K Feb  7 04:39  drive
drwxr-xr-x 3 root root 4.0K Feb  7 04:08  models
-rw-r--r-- 1 root root 5.0K Feb  7 04:50  README.md
drwxr-xr-x 1 root root 4.0K Feb  5 20:33  sample_data


### 训练完成后，第一时间保存模型权重文件。
保存模型权重

In [29]:
model_dir="models/whisper-large-v2-asr-int8/model"
model_to_save = trainer.save_model(model_dir)

In [30]:
print(model_to_save)

None


In [31]:
trainer.save_state()

In [32]:
!ls -lh models

total 4.0K
drwxr-xr-x 5 root root 4.0K Feb  7 04:48 whisper-large-v2-asr-int8


In [33]:
!ls -lh models/whisper-large-v2-asr-int8

total 20M
-rw-r--r-- 1 root root  720 Feb  7 04:34 adapter_config.json
-rw-r--r-- 1 root root  16M Feb  7 04:34 adapter_model.safetensors
-rw-r--r-- 1 root root  34K Feb  7 04:44 added_tokens.json
-rw-r--r-- 1 root root 483K Feb  7 04:44 merges.txt
drwxr-xr-x 2 root root 4.0K Feb  7 04:47 model
-rw-r--r-- 1 root root  52K Feb  7 04:44 normalizer.json
-rw-r--r-- 1 root root 5.0K Feb  7 04:34 README.md
drwxr-xr-x 3 root root 4.0K Feb  7 04:08 runs
-rw-r--r-- 1 root root 2.1K Feb  7 04:44 special_tokens_map.json
drwxr-xr-x 2 root root 4.0K Feb  7 04:45 tokenizer
-rw-r--r-- 1 root root 277K Feb  7 04:44 tokenizer_config.json
-rw-r--r-- 1 root root 2.4M Feb  7 04:44 tokenizer.json
-rw-r--r-- 1 root root 1.1K Feb  7 04:48 trainer_state.json
-rw-r--r-- 1 root root 816K Feb  7 04:44 vocab.json


In [34]:
!ls -lh models/whisper-large-v2-asr-int8/trainer_state.json

-rw-r--r-- 1 root root 1.1K Feb  7 04:48 models/whisper-large-v2-asr-int8/trainer_state.json


### 使用 Pipiline 加载 LoRA 模型，实现自动语音识别任务

In [ ]:
%mkdir -p data/audio

In [25]:
test_audio = "data/audio/test_zh.flac"

In [43]:
from transformers import AutomaticSpeechRecognitionPipeline

pipeline = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)

forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task=task)

The model 'PeftModel' is not supported for . Supported models are ['Pop2PianoForConditionalGeneration', 'SeamlessM4TForSpeechToText', 'SpeechEncoderDecoderModel', 'Speech2TextForConditionalGeneration', 'SpeechT5ForSpeechToText', 'WhisperForConditionalGeneration', 'Data2VecAudioForCTC', 'HubertForCTC', 'MCTCTForCTC', 'SEWForCTC', 'SEWDForCTC', 'UniSpeechForCTC', 'UniSpeechSatForCTC', 'Wav2Vec2ForCTC', 'Wav2Vec2ConformerForCTC', 'WavLMForCTC'].


In [44]:
with torch.cuda.amp.autocast():
    text = pipeline(test_audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [45]:
text

'这是一段测试用于WhisperLarge V2模型的自动语音识别测试。'

#### 测试自己的声音

In [48]:
test_audio = "data/audio/test_zh_mine.mp3"
# 阅读以下内容录制：
# 这段测试旨在评估WhisperLarge V2模型的自动语音识别性能。
# 通过分析其在各种语音输入情境下的表现，我们可以更全面地了解其准确性和可靠性。
# 这对于improving speech recognition technology的质量和适用性至关重要，
# especially in modern intelligent devices和virtual assistants的应用中。

In [ ]:
from transformers import AutomaticSpeechRecognitionPipeline

pipeline = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)

forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task=task)

In [49]:
with torch.cuda.amp.autocast():
    text = pipeline(test_audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [50]:
text

'这段测试只在评估WhisperLatch V2模型的自动语音识别性能，通过分析其在各种语音输入情景下的表现，我们可以更全面地了解其准确性和可靠性，这对于improving speech recognition technology的质量和适用性至关重要，especially in modern intelligent device和virtual assistance的应用。'

#### Homework 1: 为中文语料的训练过程增加过程评估，观察 Train Loss 和 Validation Loss 变化；
#### Homework 2: LoRA 模型训练完成后，使用测试集进行完整的模型评估

## 评估模型

In [39]:
import evaluate

# 词错误率（WER）是评估ASR模型常用的指标。从 Evaluate加载 WER 指标
metric = evaluate.load("wer")

In [40]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)

model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): WhisperForConditionalGeneration(
      (model): WhisperModel(
        (encoder): WhisperEncoder(
          (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
          (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
          (embed_positions): Embedding(1500, 1280)
          (layers): ModuleList(
            (0-31): 32 x WhisperEncoderLayer(
              (self_attn): WhisperAttention(
                (k_proj): Linear8bitLt(in_features=1280, out_features=1280, bias=False)
                (v_proj): lora.Linear8bitLt(
                  (base_layer): Linear8bitLt(in_features=1280, out_features=1280, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1280, out_features=8, bias=False)
                  )
                

In [41]:
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()

  0%|          | 0/1323 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  3%|▎         | 44/1323 [09:38<4:40:04, 13.14s/it]


KeyboardInterrupt: 

In [42]:
wer = 100 * metric.compute()
print(f"{wer=}")

wer=68.4659090909091


# 保存到Google Drive

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
%mkdir -p /content/drive/MyDrive/AI/FineTune/homework/week4-homework1/save

In [21]:
!ls -lh models

total 4.0K
drwxr-xr-x 3 root root 4.0K Feb  7 04:34 whisper-large-v2-asr-int8


In [38]:
cp -r models /content/drive/MyDrive/AI/FineTune/homework/week4-homework1/save2